In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, accuracy_score
import h5py
class MultiLinear:
    
    def __init__(self):
         # Learning Rate
        self.l_rate = 0.000001
         # Total iterations
        self.epochs = 10000
    
    
    def gradient_descent(self, x_train_data, y_train_data):
        
        w = np.random.rand(x_train_data.shape[1], 1) * 0.01
        b = np.zeros((1, 1), dtype='float')

        m = len(x_train_data)
    
        for i in range(5):
            
            ######## Forward Propagation ########
            # Hypothesis function :
            z = (np.dot(w.T, x_train_data.T)) + b  # Z = W*X = (1 x m)
            # Activation Function :
            a =  1 / 1 + np.exp(-z) # Activation function = sigmoid(-z) = (1 x m)

            ####### Backward Propagation ########
            dz = (a - y_train_data.T)  # dZ = a - y
            dw = (np.dot(x_train_data.T, dz.T)) / m # dW = X*T
            db = (np.sum(dz)) / m # db = dZ

            # Gradient Descent :
            w = w - np.dot(self.l_rate, dw) # w = w - alpha * dW
            b = b - np.dot(self.l_rate, db) # b = b - alpha * db 

        return w, b
    
    # Prediction :
    def prediction(self, parameters, x_test_data):
        z = np.dot(parameters[0].T, x_test_data.T) + parameters[1] # Z = W*X = (1 x m)
        return 1 / 1 + np.exp(-z) # Activation function = sigmoid(-z) = (1 x m)
    
    def remove_outlier(self, df_in):
        for col_name in list(df_in.columns):
            if is_numeric_dtype(df_in[col_name]):
                q1 = df_in[col_name].quantile(0.25)
                q3 = df_in[col_name].quantile(0.75)
                iqr = q3-q1 #Interquartile range
                fence_low  = q1-1.5*iqr
                fence_high = q3+1.5*iqr
                df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
        return df_out

    
def main():   
    import numpy as np
    import pandas as pd
    import matplotlib as plt
    from pandas.api.types import is_numeric_dtype
    import seaborn as sns
    
    m_l = MultiLinear()

    df_1 = pd.read_csv("Churn_Modelling.csv")

    # Drop unnecessary columns
    df_1 = df_1.drop("RowNumber",axis = 1)
    df_1 = df_1.drop("CustomerId",axis = 1)
    df_1 = df_1.drop("Surname",axis = 1)
    
    # Handle the categorical data in data set
    df = pd.get_dummies(df_1,drop_first=True)
    
    # Feature scale 
    for name in df.columns:
            if 'Exited' == name :     
                pass
            else:
                temp = 0
                temp_arr = np.array([])
                df[name] = (df[name] - df[name].mean()) / np.nanstd(df[name])

    
    y_data_set = np.array(df.iloc[:,-1]).reshape(df.shape[0], 1)
    x_data_set = np.array(df.iloc[:,:df.shape[1]-1])

    train_size = int(np.ceil((len(y_data_set) * 80) / 100))
    test_size = int(len(y_data_set) - train_size)

    x_train_data = np.array(x_data_set[:train_size])
    y_train_data = np.array(y_data_set[:train_size])
    x_test_data = np.array(x_data_set[train_size:])
    y_test_data = np.array(y_data_set[train_size:])

    parameters = m_l.gradient_descent(x_train_data, y_train_data)
    pred = m_l.prediction(parameters, x_test_data) 
    pred_train= m_l.prediction(parameters, x_train_data)  
    
    train_accuracy = (100 - np.mean(np.abs(pred_train - y_train_data)) * 100)
    test_accuracy = (100 - np.mean(np.abs(pred - y_test_data)) * 100)

    print("train_accuracy = ", train_accuracy)
    print("test_accuracy = ", test_accuracy)

if __name__ == '__main__':
    main()

train_accuracy =  -100.40949237413375
test_accuracy =  -97.54956196758567
